In [1]:
import pandas as pd

Многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:

2018-11-09 15:45:21;
11/09/2018 3:45:20 PM;
2018-11-09T15:45:21.2984.

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

In [2]:
melb_df = pd.read_csv('data/melb_data_ps.csv', sep=',')
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [8]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

Выделение атрибутов datetime

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

- date — дата;
- year, month, day — год, месяц, день;
- time — время;
- hour, minute, second — час, минута, секунда;
- dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
- day_name — название дня недели;
- dayofyear — порядковый день года;
- quarter — квартал (интервал в три месяца).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [9]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.

Из результатов видно, что данные представлены за интервал с 2016 по 2017 год и наибольшее количество объектов было продано в 2017 году.

Примечание. Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [23]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)


MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [24]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [12]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [13]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

#Примечание. Обратите внимание, что, 
#так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [14]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [28]:
#Задание 3.3
#Создайте в таблице melb_df признак WeekdaySale (день недели). 
# Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), 
#результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)
print(melb_df['Date'])

12822
0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]


In [17]:
ufo = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep = ',')
display(ufo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18241 entries, 0 to 18240
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   City             18215 non-null  object
 1   Colors Reported  2882 non-null   object
 2   Shape Reported   15597 non-null  object
 3   State            18241 non-null  object
 4   Time             18241 non-null  object
dtypes: object(5)
memory usage: 712.7+ KB


None

In [18]:
pd.to_datetime(ufo['Time'])
ufo['Time'] = pd.to_datetime(ufo.Time)
years_sold = ufo['Time'].dt.year
display(years_sold.value_counts())

Time
1999    2774
2000    2635
1998    1743
1995    1344
1997    1237
        ... 
1930       2
1941       2
1933       1
1934       1
1935       1
Name: count, Length: 68, dtype: int64

In [26]:
#Найдите средний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV).
ufo['Date'] = ufo['Time'].dt.date
final = ufo[ufo['State']=='NV']['Date'].diff().dt.days.mean()
print(ufo['Date'])
print(final)


0        1930-06-01
1        1930-06-30
2        1931-02-15
3        1931-06-01
4        1933-04-18
            ...    
18236    2000-12-31
18237    2000-12-31
18238    2000-12-31
18239    2000-12-31
18240    2000-12-31
Name: Date, Length: 18241, dtype: object
68.92932862190813


Чтобы выделить дату из столбца Time, можно воспользоваться атрибутом datetime date.

Чтобы вычислить разницу между двумя соседними датами в столбце, примените к нему метод diff().

Чтобы перевести интервал времени в дни, воспользуйтесь атрибутом timedelta days.

In [25]:
print(melb_df['WeekdaySale'])

0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: WeekdaySale, Length: 13580, dtype: int32
